In [4]:
import pandas as pd
dataset=pd.read_csv("insurance_pre.csv")
dataset

,age,sex,bmi,children,smoker,charges
0,19,female,27.900,0,yes,16884.92400
1,18,male,33.770,1,no,1725.55230
2,28,male,33.000,3,no,4449.46200
3,33,male,22.705,0,no,21984.47061
4,32,male,28.880,0,no,3866.85520
...,...,...,...,...,...,...
1333,50,male,30.970,3,no,10600.54830
1334,18,female,31.920,0,no,2205.98080
1335,18,female,36.850,0,no,1629.83350
1336,21,female,25.800,0,no,2007.94500


In [5]:
dataset=pd.get_dummies(dataset,drop_first=True)
dataset

,age,bmi,children,charges,sex_male,smoker_yes
0,19,27.900,0,16884.92400,0,1
1,18,33.770,1,1725.55230,1,0
2,28,33.000,3,4449.46200,1,0
3,33,22.705,0,21984.47061,1,0
4,32,28.880,0,3866.85520,1,0
...,...,...,...,...,...,...
1333,50,30.970,3,10600.54830,1,0
1334,18,31.920,0,2205.98080,0,0
1335,18,36.850,0,1629.83350,0,0
1336,21,25.800,0,2007.94500,0,0


In [6]:
dataset.columns

Index(['age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'], dtype='object')

In [7]:
independent=dataset[['age', 'bmi', 'children', 'sex_male', 'smoker_yes']]

In [8]:
dependent=dataset[['charges']]

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(independent,dependent,test_size=0.20,random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train=sc.fit_transform(x_train)
x_test=sc.transform(x_test)

In [20]:
import lightgbm as lgb
from lightgbm import LGBMRegressor

In [23]:
from sklearn.model_selection import GridSearchCV
grid_params= {"learning_rate":[1,10,50],
             "n_estimators":[10,50,100,1000],
              "max_depth":[3,5,9],
              "subsample":[0.3,0.5,0.9],
              "colsample_bytree":[0.3,0.7,0.9],
              "min_child_samples":[3,6,9]
             }

In [24]:
grid=GridSearchCV(LGBMRegressor(),grid_params,refit=True,verbose=3,n_jobs=-1)
grid.fit(x_train,y_train)

Fitting 5 folds for each of 972 candidates, totalling 4860 fits


C:\Users\jayac\AppData\Local\anaconda3\lib\site-packages\sklearn\utils\validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LGBMRegressor(), n_jobs=-1,
             param_grid={'colsample_bytree': [0.3, 0.7, 0.9],
                         'learning_rate': [1, 10, 50], 'max_depth': [3, 5, 9],
                         'min_child_samples': [3, 6, 9],
                         'n_estimators': [10, 50, 100, 1000],
                         'subsample': [0.3, 0.5, 0.9]},
             verbose=3)

In [29]:
result=grid.cv_results_
print("The resut is:{}".format(grid.best_params_))

The resut is:{'colsample_bytree': 0.3, 'learning_rate': 1, 'max_depth': 3, 'min_child_samples': 9, 'n_estimators': 10, 'subsample': 0.3}


In [31]:
table=pd.DataFrame.from_dict(result)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_colsample_bytree,param_learning_rate,param_max_depth,param_min_child_samples,param_n_estimators,param_subsample,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.030283,0.021912,0.004970,0.002592,0.3,1,3,3,10,0.3,"{'colsample_bytree': 0.3, 'learning_rate': 1, ...",8.554998e-01,8.248375e-01,8.076220e-01,8.228969e-01,7.862980e-01,8.194308e-01,2.271321e-02,16
1,0.018388,0.007278,0.004797,0.001718,0.3,1,3,3,10,0.5,"{'colsample_bytree': 0.3, 'learning_rate': 1, ...",8.554998e-01,8.248375e-01,8.076220e-01,8.228969e-01,7.862980e-01,8.194308e-01,2.271321e-02,16
2,0.014391,0.001742,0.004998,0.002097,0.3,1,3,3,10,0.9,"{'colsample_bytree': 0.3, 'learning_rate': 1, ...",8.554998e-01,8.248375e-01,8.076220e-01,8.228969e-01,7.862980e-01,8.194308e-01,2.271321e-02,16
3,0.022585,0.001854,0.004597,0.001742,0.3,1,3,3,50,0.3,"{'colsample_bytree': 0.3, 'learning_rate': 1, ...",8.120456e-01,7.514030e-01,7.795644e-01,8.174058e-01,7.575639e-01,7.835965e-01,2.713960e-02,61
4,0.024783,0.002134,0.003600,0.000490,0.3,1,3,3,50,0.5,"{'colsample_bytree': 0.3, 'learning_rate': 1, ...",8.120456e-01,7.514030e-01,7.795644e-01,8.174058e-01,7.575639e-01,7.835965e-01,2.713960e-02,61
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
967,0.090544,0.012761,0.008398,0.000491,0.9,50,9,9,100,0.5,"{'colsample_bytree': 0.9, 'learning_rate': 50,...",-9.035033e+70,-8.680955e+70,-8.880100e+70,-8.095253e+70,-7.777192e+70,-8.493707e+70,4.794903e+69,856
968,0.089345,0.004920,0.008395,0.000490,0.9,50,9,9,100,0.9,"{'colsample_bytree': 0.9, 'learning_rate': 50,...",-9.035033e+70,-8.680955e+70,-8.880100e+70,-8.095253e+70,-7.777192e+70,-8.493707e+70,4.794903e+69,856
969,0.148707,0.073764,0.003198,0.000399,0.9,50,9,9,1000,0.3,"{'colsample_bytree': 0.9, 'learning_rate': 50,...",-9.035033e+70,-8.680955e+70,-8.880100e+70,-8.095253e+70,-7.777192e+70,-8.493707e+70,4.794903e+69,856
970,0.089145,0.003368,0.003598,0.000489,0.9,50,9,9,1000,0.5,"{'colsample_bytree': 0.9, 'learning_rate': 50,...",-9.035033e+70,-8.680955e+70,-8.880100e+70,-8.095253e+70,-7.777192e+70,-8.493707e+70,4.794903e+69,856


In [32]:
'age', 'bmi', 'children', 'charges', 'sex_male', 'smoker_yes'
age=int(input("Enter the Age: "))
bmi=int(input("Enter the BMI_range: "))
children=int(input("Enter the Children_count: "))
sex_male=int(input("Enter the sex, male 1/female 0: "))
smoker_yes=int(input("Enter the smoker, yes 1/no 0): "))

pred=grid.predict([[age,bmi,children,sex_male,smoker_yes]])
print("Prediction is: {}".format(pred))

Enter the Age: 23
Enter the BMI_range: 25
Enter the Children_count: 0
Enter the sex, male 1/female 0: 0
Enter the smoker, yes 1/no 0): 1
Prediction is: [56524.44388457]
